## Introduction

Database design is all about structuring data and organizing the relationships in the database. This is my attempt at trying my hand on it! I will be using a bank marketing dataset from [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/222/bank+marketing). 

I will be using `pandas` to clean and store the data from this particular bank that they collected from a marketing campaign that aims to get customers to take out a personal loan. In this scenario, I'm assuming that the bank plans to utilize the data in the future and would like to store it. Thus, I will be setting up a `PostgreSQL database` to store this campaign's data, and designing the schema in a way that future campaigns can be easily stored and imported.

I will be also trying out a database design website called [dbdiagram.io](https://dbdiagram.io/) that generates a UML database diagram based on the SQL schema you input.

# Defining and documenting the database schemas

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [1]:
import pandas as pd
import numpy as np

# Read in the data
data = pd.read_csv("bank_marketing.csv")

### Exploratory Data Analysis

In [2]:
print(data.columns)

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'housing', 'loan', 'contact', 'month', 'day', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')


In [3]:
data.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day             41188 non-null  int64  
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

In [5]:
data['pdays']

0        999
1        999
2        999
3        999
4        999
        ... 
41183    999
41184    999
41185    999
41186    999
41187    999
Name: pdays, Length: 41188, dtype: int64

In [6]:
#Client DF
#Needs id, age, job, marital, education, credit_default, housing, loan
client_cols = ['client_id', 'age', 'job', 'marital', 'education', 'credit_default','housing', 'loan']
client_df = data[client_cols]
client_df.head()

,client_id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [7]:
#Campaign DF
#Needs `campaign_id` ,`client_id`,`number_contacts`,`contact_duration`  ,`pdays`, `previous_campaign_contacts`  ,`previous_outcome`  ,`campaign_outcome` , `last_contact_date`
campaign_cols = ['client_id', 'campaign', 'duration', 'pdays', 'previous', 'poutcome', 'y', 'month', 'day']
campaign_df = data[campaign_cols]
campaign_df.head()

,client_id,campaign,duration,pdays,previous,poutcome,y,month,day
0,0,1,261,999,0,nonexistent,no,may,13
1,1,1,149,999,0,nonexistent,no,may,19
2,2,1,226,999,0,nonexistent,no,may,23
3,3,1,151,999,0,nonexistent,no,may,27
4,4,1,307,999,0,nonexistent,no,may,3


In [8]:
#Economics DF
#`client_id`,`emp_var_rate`,`cons_price_idx`,`euribor_three_months`,
#,`number_employed` 
economics_cols = ['client_id','emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed', 'y']
economics_df = data[economics_cols]
economics_df.head()

,client_id,emp_var_rate,cons_price_idx,euribor3m,nr_employed,y
0,0,1.1,93.994,4.857,5191.0,no
1,1,1.1,93.994,4.857,5191.0,no
2,2,1.1,93.994,4.857,5191.0,no
3,3,1.1,93.994,4.857,5191.0,no
4,4,1.1,93.994,4.857,5191.0,no


### Cleaning

In [9]:
# Client_DF
client_df = client_df.rename(columns = {'client_id' : 'id'})
client_df['education'] = client_df['education'].str.replace('\.', '_', regex = True)
client_df['education'] = client_df['education'].replace('unknown', np.nan)
client_df['job'] = client_df['job'].str.replace('\.', '', regex = True)
client_df.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [10]:
# Campaign_DF
campaign_df = campaign_df.rename(columns = {'duration' : 'contact_duration', 'previous' : 'previous_campaign_contacts', 'y' : 'campaign_outcome', 'poutcome': 'previous_outcome', 'campaign' : 'number_contacts'})
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].replace({'success': 1, 'failure': 0, 'nonexistent' : np.nan})
campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].replace({'success': 1, 'failure': 0})
campaign_df.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day
0,0,1,261,999,0,NaN,no,may,13
1,1,1,149,999,0,NaN,no,may,19
2,2,1,226,999,0,NaN,no,may,23
3,3,1,151,999,0,NaN,no,may,27
4,4,1,307,999,0,NaN,no,may,3


In [11]:
# Economics_DF
economics_df = economics_df.rename(columns = {'euribor3m' : 'euribor_three_months', 'nr_employed' : 'number_employed'})
economics_df.head()

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed,y
0,0,1.1,93.994,4.857,5191.0,no
1,1,1.1,93.994,4.857,5191.0,no
2,2,1.1,93.994,4.857,5191.0,no
3,3,1.1,93.994,4.857,5191.0,no
4,4,1.1,93.994,4.857,5191.0,no


In [12]:
# More Cleaning on Campaign_DF
campaign_df['campaign_id'] = 1
cols = campaign_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
campaign_df = campaign_df[cols]
campaign_df['month'] = campaign_df['month'].str.lower().replace({'jan' : 1, 'feb': 2, 'mar': 3, 'apr' : 4, 'may' : 5, 'jun' : 6, 'jul' : 7, 'aug': 8, 'sep' : 9, 'oct' : 10, 'nov' : 11, 'dec': 12})
campaign_df['last_contact_date'] = pd.to_datetime(dict(year = 2022, month = campaign_df.month, day = campaign_df.day))
campaign_df = campaign_df.drop(columns = ['month', 'day'])
campaign_df.head()

,campaign_id,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,1,0,1,261,999,0,NaN,no,2022-05-13
1,1,1,1,149,999,0,NaN,no,2022-05-19
2,1,2,1,226,999,0,NaN,no,2022-05-23
3,1,3,1,151,999,0,NaN,no,2022-05-27
4,1,4,1,307,999,0,NaN,no,2022-05-03


### Saving

In [13]:
# Save dataframes to csv files
client_df.to_csv("client.csv", index = False)
campaign_df.to_csv("campaign.csv", index = False)
economics_df.to_csv("economics.csv", index = False)

### SQL Tables

In [14]:
# SQL Code to create tables
client_table = """CREATE TABLE client(
                 id SERIAL PRIMARY KEY,
                 age INTEGER,
                 job TEXT
                 marital TEXT
                 education TEXT
                 credit_default BOOLEAN
                 housing BOOLEAN
                 loan BOOLEAN);
                 \copy client from 'client.csv' DELIMITER ',' CSV HEADER
                 """

In [15]:
campaign_table = """CREATE TABLE campaign(
                 campaign_id SERIAL PRIMARY KEY,
                 client_id SERIAL REFERENCES client (id),
                 number_contacts INTEGER,
                 contact_duration INTEGER,
                 pdays INTEGER, 
                 previous_campaign_contacts INTEGER,
                 previous_outcome BOOLEAN,
                 campaign_outcome BOOLEAN,
                 last_contact_date DATE);
                 \copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
                 """

In [16]:
economics_table = """CREATE TABLE economics(
                 client_id SERIAL REFERENCES client (id),
                 emp_var_rate FLOAT,
                 cons_price_idx FLOAT,
                 euribor_three_months FLOAT,
                 number_employed FLOAT);
                 \copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
                 """